### Topological Analysis Notebook

**Author: Joseph Turner (jiturner@bwh.harvard.edu)**

This notebook identifies the local extrema (minima and maxima) of a continuous 3d NIfTI volume.  
It also uses linear regression to represent the NIfTI volume as a linear combination of the connectivity profiles at each local extremum.

Returns:  
- `topological_results.csv`: a CSV file describing the results of the topological analysis
    - Location, regression coefficient and individual correlation to the original NIfTI volume for each local extremum
- `regression_map.nii.gz`: A NIfTI volume representing the linear combination of the connectivity profiles at each local extremum, designed to approximate the original NIfTI volume

Other important variables:  
- `df`: Dataframe representing the results of the topological analysis
- `r_squared`: Variable containing the R^2 value of the linear regression

#### 1. Determine Local Extrema  
Optional: Replace `order=10` with the desired radius of the local extrema search. The smaller the order, the more local extrema will be found.

In [ ]:
from nifti_utils.NiftiHandler import NiftiHandler as nh
from nifti_utils.NiftiHandler import Coordinate2mm as c2mm
import pandas as pd

# Path to the NIfTI file
path='/PHShome/jt041/projects/takotsubo/PALM_analysis/all_with_mgh(n=72)/_vox_tstat.nii'

# Determine the local extrema
img = nh.load(path).apply_anatomical_mask()

df = img.get_local_extrema(order=10) # Order is the number of voxels to consider in each direction

df['coord_2mm'] = df.apply(lambda x: c2mm((x['x'],x['y'], x['z']), 'voxel'), axis=1)
df['mni_coord'] = df['coord_2mm'].apply(lambda x: x.mni_space_coord)
df['anatomical_name'] = df['coord_2mm'].progress_apply(lambda x: x.anatomical_name)
df['path'] = '/data/nimlab/precomputed_connectomes/GSP1000_MF/' + df['x'].astype(str) + '_' + df['y'].astype(str) + '_' + df['z'].astype(str) + '_T.nii.gz'
df['x'] = df['mni_coord'].apply(lambda x: x[0])
df['y'] = df['mni_coord'].apply(lambda x: x[1])
df['z'] = df['mni_coord'].apply(lambda x: x[2])
df['image'] = df['path'].progress_apply(nh.load)
df.drop(columns=['coord_2mm', 'mni_coord'], inplace=True)

#### 2. Linear Regression on Local Extrema Voxel Connectivity
Optional: Set `feature_selection = True` to select a specific number of local extrema to use in the linear regression model.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import nibabel as nib
from nifti_utils.AnalysisUtils import ImageComparison

# Extract data arrays
X = np.vstack(df['image'].apply(lambda x: x.data.flatten())).T
y = img.data.flatten()

# Optional feature selection
feature_selection = True  # Change to True to select specific extrema to use as features

if feature_selection:
    total_features = X.shape[1]
    
    no_of_correlated_voxels = 10 # Replace with desired number of correlated voxels to use in the model    
    no_of_anticorrelated_voxels = 5 # Replace with desired number of anticorrelated voxels to use in the model

    no_of_correlated_voxels = min(no_of_correlated_voxels, len(df[df['value']>0]))
    first_part_indices = np.arange(no_of_correlated_voxels)
    no_of_anticorrelated_voxels = min(no_of_anticorrelated_voxels, len(df[df['value']<0]))
    last_part_indices = np.arange(total_features - no_of_anticorrelated_voxels, total_features)
    selected_features_indices = np.concatenate((first_part_indices, last_part_indices))
    
    # Select features based on these indices
    X = X[:, selected_features_indices]

X = np.nan_to_num(X)

# Train the model
model = LinearRegression()
model.fit(X, y)
r_squared = model.score(X, y)

# Extract model coefficients
coefficients = model.coef_

if feature_selection:
    full_coefficients = np.zeros(total_features)
    full_coefficients[selected_features_indices] = model.coef_
else:
    full_coefficients = model.coef_

df['coefficient'] = full_coefficients

df = df[df['coefficient'] != 0].reset_index(drop=True)

# Correlate the original image with the selected images

comparer = ImageComparison()
img_fake = nh.load('/data/nimlab/dl_archive/takutsuboLesions_GSP1000_V3/sub-01an2020/connectivity/sub-01an2020_tome-GSP1000uMF_space-2mm_stat-avgRFz_conn.nii.gz')
df['images_to_correlate'] = df['image'].apply(lambda x: x.copy())
correlation_df = comparer.correlate_image_with_list_as_df(img.copy(), df['images_to_correlate'].copy(deep=True).tolist()) # This is the line that is causing the issue
df['correlation'] = correlation_df['correlation']
df.drop(columns=['images_to_correlate'], inplace=True)

# Generate the combined image
original_shape = img.shape
combined_image_data = np.dot(X, df[df['coefficient']!=0]['coefficient'].values)
combined_image = combined_image_data.reshape(original_shape)

print(f'R-squared: {r_squared}')
display(df)

#### 3. Save Results (CSV and NIfTI)

In [ ]:
regression_image = nh.load(combined_image)
regression_image.nifti_obj.to_filename('regression_map.nii.gz') # Replace with desired output filename

df.copy().drop(columns=['image']).to_csv('topological_results.csv', index=False)
display(df)